In [39]:
import numpy as np
from numpy import linalg as LA
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic
from tqdm import trange
from numba import njit
import time

%load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [40]:
def compute_reprojection_error(P1, P2, Xj, x1j, x2j):

    x1j_proj = cv.transform_and_dehomogenize(P1, Xj)
    x2j_proj = cv.transform_and_dehomogenize(P2, Xj)
    
    r1 = np.array([x1j[0] - x1j_proj[0], x1j[1] - x1j_proj[1]])
    r2 = np.array([x2j[0] - x2j_proj[0], x2j[1] - x2j_proj[1]])
    res = np.concatenate((r1, r2), 0)
    reproj_err = LA.norm(res)**2

    return reproj_err, res

In [41]:
def compute_total_reprojection_error(P1, P2, X_est, x1, x2):

    n_pts = np.size(X_est,1)
    reproj_err_tot = []
    res_tot = []

    for j in range(n_pts):

        Xj = X_est[:,j]
        x1j = x1[:,j]
        x2j = x2[:,j]

        reproj_err, res = compute_reprojection_error(P1, P2, Xj, x1j, x2j)
        reproj_err_tot.append(reproj_err)
        res_tot.append(res)
    
    res_tot = np.concatenate(res_tot, 0)

    print('\nTotal reprojection error:', round(np.sum(reproj_err_tot), 2))
    print('Median reprojection error:', round(np.median(reproj_err_tot), 2))
    print('Avg. reprojection error:', round(np.mean(reproj_err_tot), 2))

    return reproj_err_tot, res_tot

In [42]:
def compute_jacobian_of_r(Pi, Xj):
    J1 = (Pi[-1,:] * (Pi[0,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - (Pi[0,:] / (Pi[-1,:] @ Xj))
    J2 = (Pi[-1,:] * (Pi[1,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - (Pi[1,:] / (Pi[-1,:] @ Xj))
    J = np.row_stack((J1, J2))
    return J

In [43]:
def linearize_reprojection_error(P1, P2, Xj, x1j, x2j):

    _, r = compute_reprojection_error(P1, P2, Xj, x1j, x2j)

    J1 = compute_jacobian_of_r(P1, Xj)
    J2 = compute_jacobian_of_r(P2, Xj)
    J = np.row_stack((J1, J2))
    
    return r, J

In [44]:
def compute_update(r, J, mu):
    I = np.eye(np.size(J,1))
    delta_Xj = -LA.inv(J.T @ J + mu*I) @ J.T @ r
    return delta_Xj

In [45]:
def optimize_X(P1, P2, X, x1, x2, mu_init, n_its):

    n_pts = np.size(X,1)
    ts = []

    for j in trange(n_pts):
        
        Xj = X[:,j]
        x1j = x1[:,j]
        x2j = x2[:,j]

        t = 0
        converged = False
        mu = mu_init
    
        while (t <= n_its) and converged is not True:
        
            t += 1
            r, J = linearize_reprojection_error(P1, P2, Xj, x1j, x2j)
            delta_Xj = compute_update(r, J, mu)
            Xj_opt = cv.dehomogenize(Xj + delta_Xj)
            
            reproj_err, _ = compute_reprojection_error(P1, P2, Xj, x1j, x2j)
            reproj_err_opt, _ = compute_reprojection_error(P1, P2, Xj_opt, x1j, x2j)

            if np.isclose(reproj_err_opt, reproj_err):
                converged = True
            elif reproj_err_opt < reproj_err:
                Xj = Xj_opt
                mu /= 10
            else:
                mu *= 10
        
        X[:,j] = Xj
        ts.append(t)

    print('\nAvg its:', np.mean(ts))
    print('Max its:', np.max(ts))
    print('Min its:', np.min(ts))

    return X

In [46]:
def plot_cameras_and_3D_points(X_est, X_opt, C_arr, axis_arr, s, path, save=False):
    
    fig = plt.figure(figsize=(8,6))
    ax = plt.axes(projection='3d')

    ax.plot(X_est[0], X_est[1], X_est[2], '.', ms=5, color='magenta', label='Est. X')
    ax.plot(X_opt[0], X_opt[1], X_opt[2], '.', ms=1.7, color='lime', label='Opt. X')
    cv.plot_cameras_and_axes(ax, C_arr, axis_arr, s)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.set_aspect('equal')
    ax.view_init(elev=-50, azim=-104, roll=20)
    ax.legend(loc="lower right")
    fig.tight_layout()
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [47]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-4'
data = r'\data-2023\data'
compex3 = r'\compEx3data.mat'
sift_matlab = r'\CE2_sift_matlab.mat'
report = r'\report-images'

eriks_sol = True
plt_3D = False
save = False

if eriks_sol:
    ind = 2

    x1 = cv.homogenize(cv.convert_mat_to_np(path+data+sift_matlab, 'xA'), multi=True)
    x2 = cv.homogenize(cv.convert_mat_to_np(path+data+sift_matlab, 'xB'), multi=True)
    inliers = np.load(path+data+'/CE2_inliers_{}.npy'.format(ind))
    x1 = x1[:,inliers]
    x2 = x2[:,inliers]
    X_est = np.load(path+data+'/CE2_X_valid_{}.npy'.format(ind))

    K = cv.convert_mat_to_np(path+data+'/compEx2data.mat', 'K')
    P1 = cv.transform(K, cv.get_canonical_camera())
    P2 = cv.transform(K, np.load(path+data+'/CE2_P2_valid_{}.npy'.format(ind)))
else:
    ind = 3
    
    x = cv.convert_mat_to_np(path+data+compex3, 'x')
    x1 = x[0,0]
    x2 = x[0,1]
    X_est = cv.convert_mat_to_np(path+data+compex3, 'X')

    P = cv.convert_mat_to_np(path+data+compex3, 'P')
    P1 = P[0,0]
    P2 = P[0,1]

mu = 1
n_its = 100

reproj_err_tot_est, res_tot_est = compute_total_reprojection_error(P1, P2, X_est, x1, x2)
X_opt = optimize_X(P1, P2, X_est, x1, x2, mu, n_its)
reproj_err_tot_opt, res_tot_opt = compute_total_reprojection_error(P1, P2, X_opt, x1, x2)    


Total reprojection error: 3420.75
Median reprojection error: 1.5
Avg. reprojection error: 1.86


100%|██████████| 1840/1840 [00:03<00:00, 508.87it/s]



Avg its: 1.9945652173913044
Max its: 2
Min its: 1

Total reprojection error: 3250.65
Median reprojection error: 1.44
Avg. reprojection error: 1.77


In [48]:
P_arr = np.array([P1, P2])
C_arr, axis_arr = cv.compute_camera_and_normalized_principal_axis(P_arr, multi=True)
s = 1
plot_cameras_and_3D_points(X_est, X_opt, C_arr, axis_arr, s, path+report+'/CE3_3D_{}.png'.format(ind), save)